<a href="https://colab.research.google.com/github/fernanvilla/UNALTECOCO-2020-01/blob/master/Tecoco_Clase%2005_01_An%C3%A1lisis%20L%C3%A9xico%20C%C3%B3digo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# La Clase Token

In [1]:
import re
import sys

class Token(object):
    # Estructura simple de token
    def __init__(self, type, val, pos):
        # Constructor de la clase        
        self.type = type
        self.val = val
        self.pos = pos
    
    def __str__(self):
      #Método para mostrar strings, similar a ToString
      return '%s | %s |-> en la posición: %s' % (self.type, self.val, self.pos)


# La Clase LexerError

In [2]:
class LexerError(Exception):
    # Lanzador de Excepciones en el Lexer.         
    def __init__(self, pos):
        # Posición en la línea de entrada donde se generó el error
        self.pos = pos

# La Clase Lexer, Lexer Simple basado en expresiones regulares

El Contructor del Lexer tiene como parametros: 
* **`rules`**: Es un listado de reglas, cada regla es un par `regex, type`, donde `regex` es la expresión regular usada para reconocer un token y `type` es el tipo de token a retonar cuando este se ha reconocido.
* **`skip_whitespace`**: Si es True, se omitiran los espacios en blanco (\s+) y no serán analizados por el lexer. En caso de que sea False se deben especificar las reglas de cómo analizar los espacios en blanco.

In [3]:
class Lexer(object):
    def __init__(self, rules, skip_whitespace=True):
        idx = 1
        regex_parts = [] #Lista
        self.group_type = {} #Diccionario
        # Las expresiones regulares se agrupan, GROUP1, GROUP2 y asi 
        # sucesivamente, para facilitar su análisis

        for regex, type in rules:
            groupname = 'GROUP%s' % idx
            regex_parts.append('(?P<%s>%s)' % (groupname, regex))
            print('Reconocido: (?P<%s>%s)' % (groupname, regex))
            self.group_type[groupname] = type            
            idx += 1

        print("\nGrupos Reconocidos: %s" % self.group_type)
        self.regex = re.compile('|'.join(regex_parts))
        print("\n Expresión Total: %s\n" % self.regex)
        self.skip_whitespace = skip_whitespace
        self.re_ws_skip = re.compile('\S')

    def input(self, buf):
        # Inicializa el lexer con un buffer como entrada
        # el cual es la línea de caracteres a analizar        
        self.buf = buf
        self.pos = 0

    def token(self):
        # Retorna el siguiente token encontrado en el buffer de entrada.
        #    Cuando se alcanza el final del buffer se retorna None (null).
        #    Se lanza una excepción si ocurre un error lexico, es decir, 
        #    la parte actual del buffer no coincide con ninguna regla.        
      
        if self.pos >= len(self.buf):
            return None #Si ya se terminó la línea
        else:
            if self.skip_whitespace:
                m = self.re_ws_skip.search(self.buf, self.pos)

                if m:
                    self.pos = m.start()
                else:
                    return None
         
            m = self.regex.match(self.buf, self.pos) #si alguna regla aplica
            if m:
                groupname = m.lastgroup
                tok_type = self.group_type[groupname]
                tok = Token(tok_type, m.group(groupname), self.pos)
                self.pos = m.end()
                return tok

            raise LexerError(self.pos)

    def tokens(self):
        # Retorna un iterador para el token encontrado
        while 1:
            tok = self.token()
            if tok is None: break
            yield tok
        

# Uso del Lexer

In [6]:
if __name__ == '__main__':
    reglas = [        
        ('[+-]?\d+(\.\d+)?','NUMERODECIMAL'),
        ('[+-]?\d+',        'NUMEROENTERO'),
        ('seno',            'FNSENO'),
        ('[a-zA-Z_]\w+',    'IDENTIFICADOR'),
        ('\+',              'MAS'),
        ('\-',              'MENOS'),
        ('\*',              'MULTIPLICAR'),
        ('\/',              'DIVIDIR'),
        ('\(',              'PIZQ'),
        ('\)',              'PDER'),
        (':=',              'ASIGNAR'),
    ]

    #Creamos una instancia de un lexer
    lx = Lexer(reglas, skip_whitespace=True)
    #Ejemplos / casos de entradas
    lx.input('ab := (s1 + b2) / (-1.2*A1*C1)')
    #lx.input('12.4')
    #lx.input('ar := seno(20) + 56 * AB')
    #lx.input('rw*9')
    #lx.input('miformula := seno(10 * seno(15 + AB))');

    try:
        for tok in lx.tokens():
            print(tok)
    except LexerError as err:
        print('Ha ocurrido un error en la posición: %s' % err.pos)

Reconocido: (?P<GROUP1>[+-]?\d+(\.\d+)?)
Reconocido: (?P<GROUP2>[+-]?\d+)
Reconocido: (?P<GROUP3>seno)
Reconocido: (?P<GROUP4>[a-zA-Z_]\w+)
Reconocido: (?P<GROUP5>\+)
Reconocido: (?P<GROUP6>\-)
Reconocido: (?P<GROUP7>\*)
Reconocido: (?P<GROUP8>\/)
Reconocido: (?P<GROUP9>\()
Reconocido: (?P<GROUP10>\))
Reconocido: (?P<GROUP11>:=)

Grupos Reconocidos: {'GROUP1': 'NUMERODECIMAL', 'GROUP2': 'NUMEROENTERO', 'GROUP3': 'FNSENO', 'GROUP4': 'IDENTIFICADOR', 'GROUP5': 'MAS', 'GROUP6': 'MENOS', 'GROUP7': 'MULTIPLICAR', 'GROUP8': 'DIVIDIR', 'GROUP9': 'PIZQ', 'GROUP10': 'PDER', 'GROUP11': 'ASIGNAR'}

 Expresión Total: re.compile('(?P<GROUP1>[+-]?\\d+(\\.\\d+)?)|(?P<GROUP2>[+-]?\\d+)|(?P<GROUP3>seno)|(?P<GROUP4>[a-zA-Z_]\\w+)|(?P<GROUP5>\\+)|(?P<GROUP6>\\-)|(?P<GROUP7>\\*)|(?P<GROUP8>\\/)|(?P<GROUP9>\\()|(?P<GROUP10>\\))|(?P<G)

IDENTIFICADOR | ab |-> en la posición: 0
ASIGNAR | := |-> en la posición: 3
PIZQ | ( |-> en la posición: 6
IDENTIFICADOR | s1 |-> en la posición: 7
MAS | + |-> en la posició